# Clutter Overlay Dataset Creation
Alex Denton, 25 Aug 2022, Thesis work

This file overlays cropped SAR objects onto MSTAR clutter backgrounds using OpenCV and Numpy.

<b>Inputs:</b>
1. SAR Image Directory
 - Chips and Clutter Scenes
 - grayscale, 8-bit

<b>Outputs:</b><br>
1. Clutter with overlays
 - same density of overlays

<b>Notes:</b><br>
1. The noise is recalculated at each resolution, not simply downsampled.
2. The clutter patches need to be randomly sampled from the clutter dataset.

## Imports

In [ ]:
import os
import math
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from randowm import randint
from tqdm import tqdm  # for progress bar

# Custom Functions
from plotting import MultiPlots, OverContour, LayersPlot
from SarClass import *
from SupportFunctions import *

In [ ]:
debug = False

***

## Parameters
What specifics are will define this dataset?

In [ ]:
ObjOfInterest = ['m35','2s1','m1']
Resolution = 0.5  # m
jit_step = 2  # step size for jitter
jit_square = 3  # 3^2 images after jitter data augmentation
                 

***

## Define Directories

### Testing Paths (relative)

In [ ]:
# chip_dir  = 'E:\PycharmProjects/ChipOverlay/test_images/SAMPLE_OpenCV/images/bmp2'
chip_dir = [fr"/test_images/SAMPLE_OpenCV/images/{ooi}" for ooi in ObjOfInterest]
#chip_dir = [r'/test_images/SAMPLE_OpenCV/images/m35',
#            r'/test_images/SAMPLE_OpenCV/images/2s1',
#            r'/test_images/SAMPLE_OpenCV/images/m1']
annot_dir = '/test_images/SAMPLE_OpenCV/annotations'
coco_file = 'sample_synthetic_coco_format_sample1_converted.json'
out_dir = r'/test_images/overlay_clutter'
clutter_dir = r'/test_images/Clutter'

### Production Paths (relative)

### Ensure 'out_dir' exists

In [ ]:
if not os.path.exists(f'{out_dir}/'):
    os.mkdir(os.path.join(out_dir))

***

## Importing Images

### Chips
Build list of all png file names in chip_dir, but drop the ".png" b/c this will be the list of object names

In [ ]:
for m in len(chip_dir)  # import multiple chip types for simultaneous integration
    os.chdir(chip_dir[m])  # change to chip_dir
    chips = [_chip.split('.')[0] for _chip in os.listdir() if _chip.endswith(".png") and not _chip.startswith('.')]  # 'chips' is list of str

    # Create SarImage/SampleChip instances for each image
    chips = [SampleChip(f'{chip_dir[m]}/{_chip}.png') for _chip in chips]  # 'chips' is list of objects
    print(f'{len(chips)} chip objects created')


### Clutter
Also load and treat clutter for the clutter background

In [ ]:
os.chdir(clutter_dir)  # change to clutter_dir
clutters = [_clutter.split('.')[0] for _clutter in os.listdir() if _clutter.endswith(".png")]  # 'clutters' is list of str
clutters = [MstarClutter(f'{clutter_dir}/{_clutter}.png') for _clutter in clutters]  # 'clutters' is list of objects

***

## Create Masks and Histograms

In [ ]:
###### AFRL Given Masks #######
# # Get mask info for each new chip and calculate histograms
# for _chip in tqdm(chips, total=len(chips), desc="Importing Coco masks"):
#     os.chdir(annot_dir)  # change to annotation directory
#     keep1 = _chip.mask_coco(coco_file, debug=debug)  # returns 'True' if needs to be deleted
#     if keep1:
#         keep2 = _chip.histComb()  # calculate object_mean, shadow_mean, and background_mean
#         if not keep2:
#             # print(f'Masks missing for {_chip.name}')
#             pass
#     else:
#         # print(f'cannot find {_chip.name}\nremoving from list...')
#         pass
#     if not keep1 or not keep2:  # if either is 'True' then delete
#         chips.remove(_chip)  # remove from list
#         del _chip  # delete object instance from memory
# print(f'Masks uploaded / Histograms calculated for {len(chips)} chips')

### Threshold Masks - Chips
Using <i>.mask_threshold()</i> because the AFRL mask dataset was incomplete.<br>
<b>HistComb</b> assigns .shadow_mean and .background_mean to each image.

In [ ]:
for _chip in chips:
    # create masks based on thresholding
    _chip.mask_threshold(bins=256, blur_kernel=[7,7], object_offset=1, shadow_offset=0.33, debug=False)
    # assign .shadow_mean and .background_mean to each image in question
    _chip.histComb(bins = 256, debug=False)
print(f'Masks created / Histograms calculated for {len(chips)} chip images')

In [ ]:
# [_chip.feather(shadow_dilation=3, debug=debug) for _chip in chips]  # generates chip.comb_alpha
# [_chip.feather(kernel_sz=(3,3), shadow_dilation=3, debug=debug) for _chip in tqdm(chips, total=len(chips), desc='Feathering')]  # generates chip.comb_alpha

<b>Feather</b> runs dilation and Gaussian blur routines to blend the overlay. Routine found in <i>SarClass.py</i>

In [ ]:
for _chip in tqdm(chips, total=len(chips), desc='Feathering'):
    keep = _chip.feather(kernel_sz=(3,3), shadow_dilation=0, debug=debug)
    if not keep:
        # print(f'no shadow_mask for {_chip.name}\nid number {_chip.image_id}')
        chips.remove(_chip)
        del _chip
print(f'Feathering complete for {len(chips)} chips')

### Threshold Masks - Clutter
The function <i><b>.MstarClutter()</b></i> aready ran <i>.mask_threshold()</i> because there are no provided mask for this set.<br>
<b>HistComb</b> assigns .shadow_mean and .background_mean to each image.

In [ ]:
[_clutter.histComb() for _clutter in tqdm(clutters, total=(len(clutters)), desc="Preparing Clutter")]
print(f'Masks created / Histograms calculated for {len(clutters)} background clutter images')

### Data Augmentation
<b>Jitter</b> adds 9-point movement to provide variability to the dataset (data augmentation). <br>
Note that <i>.jitter()</i> returns a list of objects. Each objects is a new instance of the original chip with only 'name', 'original', and 'comb_alpha' modified.

In [ ]:
def _jitter(sets,step=jit_step,square=jit_square):
    set_jit = []
    
    for _set in tqdm(sets, total=len(set), desc=f'Jittering {sets}'):
        keep = _set.jitter(step,square,debug=False)
        if not keep:  # if returned False for error
            print(f"No comb_alpha for {_set.name}\nid {_set.image_id}")
            chips.remove(_set)
            del _set
        set_jit += keep
    
    print(f'Jittering complete for {len(sets)} images')
    return set_jit

In [ ]:
chips_jit = _jitter(chips)
clutters_jit = _jitter(clutters)

### Adjust resolution of chips_jit and clutters

In [ ]:
new_res = Resolution  # m

chips = [_chip.iso_down_sample(old_res=(_chip.resCross,_chip.resRange), new_res=new_res) for _chip in tqdm(chips_jit, total=len(chips_jit), desc='Downsample chips_jit 0.5')]  # .comb_alpha is [grayscale, alpha_mask, max_dilation]
clutters = [down_sample(_clutter, old_res=(_chip.resCross,_chip.resRange), new_res=new_res) for _clutter in tqdm(clutters_jit, total=len(clutters), desc='Downsample clutter 0.5')]  # .comb_alpha is [grayscale, alpha_mask, max_dilation]

del chips_jit, clutters_jit  # we're done with these and will remove them from memory

<div class="alert alert-block alert-info">
At this point we have two datasets - chips and clutter. Each is a list of objects with masks and labels. Each has been augmented by jittering and then downsampled to a lower resolution.<br><br>
Next, we will being the overlay procedure...
    </div>

<div class="alert alert-block alert-warning">
    This is as far as I have gone...
    </div>

***

## Create and Save New Data

### Determine Density and Distrobution of Chips

In [ ]:
print(f'There are {len(clutters)} clutter scenes after augmentation.\n')
print(f'There are:\n')
for n in len(ObjOfInterest):
    print(f'\t{len(chips.name(ObjOfInterest[n]))} chips of {ObjOfInterest[n]}\n')

In [ ]:
chips_per_clutter = 3  # density of THIS CHIP ONLY (will run the whole routine again to add more chip classes
[setattr(clutter, 'chip_count', 0) for clutter in clutters]  #
# Total chips that will be overlaid (at each resolution
total_overlays = len(clutters)*chips_per_clutter
# What percent of chips will actually be used?
chips_to_use = total_overlays/len(chips) # gives an int

### Overlay Chips on Clutter

Since patch is different for each image, it must be generated on-the-fly. Additionally, each patch must be histogram balanced to the chip's object/shadow. See BRV.ipynb for details on this process.


In [ ]:
# Set through each clutter scene
for _clutter in tqdm(clutters, total= len(clutters), desc = 'Overlaying chips on clutter'):
    while _clutter.chip_count < chips_per_clutter:

        # Pick a cchip image at random
        if len(chips) > 1:
            rand_chip = chips[random.randint(0, len(chips))-1 ]
        else:
            rand_chip = chips  # the last one remaining

        # Put the chip in a random place
        _clutter.overlay_clutter(rand_chip, out_dir=out_dir, debug=True)

        # Remove the used chip from the set

        '''
        # Cut a random patch and resize
        w = _chip.width  # this value is the original (without downsampling)
        h = _chip.height
        x, y = _clutter.rand_patch(w, h)
        patch = _clutter.cut_patch(x, y, w, h)
        _patch = patch.copy()  # break the connection to the original clutter image to prevent corruption
    
        # Scale Chip Histogram to Match Patch (forward)
        ps0 = ptSlope([_chip.shadow_mean, 0], [_clutter.shadow_mean, 1])
        ps1 = ptSlope([_chip.background_mean, 0], [_clutter.background_mean, 1])
    
        _int = intPt(ps0, ps1)
    
        _chip = distort(_int, _chip)
        _chip = _chip.astype(np.uint8)'''